In [ ]:
#    Copyright 2023, chen. All Rights Reserved.

#    Licensed under the Apache License, Version 2.0 (the "License");
#    you may not use this file except in compliance with the License.
#    You may obtain a copy of the License at

#        http://www.apache.org/licenses/LICENSE-2.0

#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.

# Models Api

使用 Models API 查看和访问 OpenAI 提供的预训练大语言模型。

## 环境准备

### 安装所需依赖组件

> %pip will install the package in the virtual environment where the current notebook kernel is running. 
> While !pip will install the package in the base environment. 
> If you are using Python virtual environment (as you should!), you should use %pip.

前置通过 shell 命令，安装所依赖的 python 包。

此步骤也仅需一次执行。

In [1]:
%pip install tiktoken openai pyyaml

Note: you may need to restart the kernel to use updated packages.


### 配置环境信息

该项目中，密钥配置在了根目录的 config.yaml 文件中，可替换为自己的密钥信息。

如使用 git 进行管理，请手动忽略该文件相关变更，避免信息泄露。

In [2]:
import openai
import yaml
import json
import os
import subprocess
from pprint import pprint

# 通过 subprocess 执行 shell 命令，获取 git 仓库的根目录
command = ['git', 'rev-parse', '--show-toplevel']
process = subprocess.Popen(command, stdout=subprocess.PIPE)
output, error = process.communicate()
git_root = output.decode().strip()

config_path = os.path.join(git_root, "openai-api", "config.yaml")
config = {}
with open(config_path,"r") as f:
    config = yaml.safe_load(f)
openai.api_key = config["sk"]

## Model List

列出当前可用的模型，并提供每个模型的基本信息，如所有者和可用性。

In [3]:
models = openai.Model.list()
print(models)

{
  "data": [
    {
      "created": 1651172509,
      "id": "curie-search-query",
      "object": "model",
      "owned_by": "openai-dev",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampling": true,
          "allow_search_indices": true,
          "allow_view": true,
          "created": 1695149182,
          "group": null,
          "id": "modelperm-8aqdyZaKtD3MD831mGbqh1MD",
          "is_blocking": false,
          "object": "model_permission",
          "organization": "*"
        }
      ],
      "root": "curie-search-query"
    },
    {
      "created": 1651172510,
      "id": "babbage-search-document",
      "object": "model",
      "owned_by": "openai-dev",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_s

#### Prompt：获取所有模型名称 

```
下面的 json 数据存在 models 变量中，我希望遍历获取 models 中每一个 "id"，并将结果存在 model_list 中，生成python代码：

"data": [
    {
      "created": 1649358449,
      "id": "babbage",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampling": true,
          "allow_search_indices": false,
          "allow_view": true,
          "created": 1669085501,
          "group": null,
          "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
          "is_blocking": false,
          "object": "model_permission",
          "organization": "*"
        }
      ],
      "root": "babbage"
    },
    {
      "created": 1669599635,
      "id": "text-davinci-003",
      "object": "model",
      "owned_by": "openai-internal",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampling": true,
          "allow_search_indices": false,
          "allow_view": true,
          "created": 1688551385,
          "group": null,
          "id": "modelperm-jepinXYt59ncUQrjQEIUEDyC",
          "is_blocking": false,
          "object": "model_permission",
          "organization": "*"
        }
      ],
      "root": "text-davinci-003"
    },
```


In [4]:
model_list = [model['id'] for model in models['data']]
print(model_list)

['curie-search-query', 'babbage-search-document', 'text-search-babbage-doc-001', 'babbage', 'text-babbage-001', 'text-similarity-davinci-001', 'davinci', 'davinci-similarity', 'code-davinci-edit-001', 'curie-similarity', 'code-search-babbage-code-001', 'curie-instruct-beta', 'text-search-ada-doc-001', 'davinci-instruct-beta', 'text-search-davinci-doc-001', 'text-curie-001', 'davinci-search-query', 'text-similarity-curie-001', 'text-davinci-001', 'text-search-davinci-query-001', 'text-davinci-003', 'ada-search-document', 'ada-code-search-code', 'babbage-002', 'whisper-1', 'davinci-002', 'davinci-search-document', 'curie-search-document', 'babbage-code-search-text', 'babbage-code-search-code', 'babbage-search-query', 'text-search-ada-query-001', 'code-search-ada-text-001', 'gpt-3.5-turbo-instruct-0914', 'gpt-3.5-turbo-instruct', 'gpt-3.5-turbo-0613', 'ada-search-query', 'ada-code-search-text', 'text-search-curie-query-001', 'text-davinci-002', 'code-search-babbage-text-001', 'text-embedd

## Retrieve Model

获取模型实例，提供有关模型的基本信息，例如所有者和权限设置。

以`text-davinci-003`模型为例，解释说明各项参数：

1. `created`: 这是模型创建的时间戳，单位为 Unix 时间戳（自1970年1月1日（00:00:00 GMT）以后的秒数）。
2. `id`: 这是模型的唯一标识符。在这个例子中，模型的 ID 是 "text-davinci-003"。
3. `object`: 这个字段表示的是当前对象的类型，在这个例子中，对象是 "model"，说明这个 JSON 对象是一个模型。
4. `owned_by`: 这个字段表示的是模型的所有者，在这个例子中，模型的所有者是 "openai-internal"。
5. `parent`: 这个字段表示的是模型的父模型。如果此字段为 null，那么说明这个模型没有父模型。
6. `permission`: 这个字段是一个包含多个键-值对的对象，描述了该模型的访问和使用权限。例如：
   - `allow_create_engine` 为 false，说明不允许创建引擎；
   - `allow_fine_tuning` 为 false，说明不允许对模型进行微调；
   - `allow_logprobs` 为 true，说明允许获取模型的 logprobs 输出；
   - `allow_sampling` 为 true，说明允许进行采样；
   - `allow_search_indices` 为 false，说明不允许搜索索引；
   - `allow_view` 为 true，说明允许查看模型；
   - `is_blocking` 为 false，说明此权限不是阻止操作的。
8. `root`: 这个字段表示的是模型的根模型。在这个例子中，模型的根模型是 "text-davinci-003"。

In [5]:
openai.Model.retrieve("text-davinci-003")

<Model model id=text-davinci-003 at 0x236571607c0> JSON: {
  "created": 1669599635,
  "id": "text-davinci-003",
  "object": "model",
  "owned_by": "openai-internal",
  "parent": null,
  "permission": [
    {
      "allow_create_engine": false,
      "allow_fine_tuning": false,
      "allow_logprobs": true,
      "allow_sampling": true,
      "allow_search_indices": false,
      "allow_view": true,
      "created": 1695054195,
      "group": null,
      "id": "modelperm-RgOpvCEIoYxuWP9GsHH0TMo1",
      "is_blocking": false,
      "object": "model_permission",
      "organization": "*"
    }
  ],
  "root": "text-davinci-003"
}

In [6]:
openai.Model.retrieve("gpt-3.5-turbo")

<Model model id=gpt-3.5-turbo at 0x23657160540> JSON: {
  "created": 1677610602,
  "id": "gpt-3.5-turbo",
  "object": "model",
  "owned_by": "openai",
  "parent": null,
  "permission": [
    {
      "allow_create_engine": false,
      "allow_fine_tuning": false,
      "allow_logprobs": true,
      "allow_sampling": true,
      "allow_search_indices": false,
      "allow_view": true,
      "created": 1695265691,
      "group": null,
      "id": "modelperm-ZaQNOH1GzC0MJhItvLsu7zLy",
      "is_blocking": false,
      "object": "model_permission",
      "organization": "*"
    }
  ],
  "root": "gpt-3.5-turbo"
}

## Completions API

使用 Completions API 实现各类文本生成任务

主要请求参数说明：

- **`model` (string, Required)**

  ID of the model to use. You can use the [List models](https://platform.openai.com/docs/api-reference/models/list) API to see all of your available models, or see our [Model overview](https://platform.openai.com/docs/models/overview) for descriptions of them.

- **`prompt` (string or array, Required)**
  
  The prompt(s) to generate completions for, encoded as a string, array of strings, array of tokens, or array of token arrays.

  Note that <|endoftext|> is the document separator that the model sees during training, so if a prompt is not specified the model will generate as if from the beginning of a new document.

- **`stream` (boolean or null, Optional, Defaults to false)**

  Whether to stream back partial progress. If set, tokens will be sent as data-only [server-sent events](https://developer.mozilla.org/en-US/docs/Web/API/Server-sent_events/Using_server-sent_events#event_stream_format) as they become available, with the stream terminated by a `data: [DONE]` message. [Example Python code](https://cookbook.openai.com/examples/how_to_stream_completions).

- **`max_tokens` (integer or null, Optional, Defaults to 16)**

  The maximum number of [tokens](https://platform.openai.com/tokenizer) to generate in the chat completion.

  The token count of your prompt plus max_tokens cannot exceed the model's context length. [Example Python code](https://cookbook.openai.com/examples/how_to_count_tokens_with_tiktoken) for counting tokens.

- **`temperature` (number, Optional, Defaults to 1)**

  What sampling temperature to use, between 0 and 2. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic.

  We generally recommend altering this or `top_p` but not both.

- **`top_p` (number or null, Optional, Defaults to 1)**

  An alternative to sampling with temperature, called nucleus sampling, where the model considers the results of the tokens with top_p probability mass. So 0.1 means only the tokens comprising the top 10% probability mass are considered.

  We generally recommend altering this or `temperature` but not both.

### 生成英文文本

In [7]:
data = openai.Completion.create(
    model="text-davinci-003",
    prompt="Say this is a test",
    max_tokens=7,
    temperature=0
)
print(data)

{
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\n\nThis is indeed a test"
    }
  ],
  "created": 1695548521,
  "id": "cmpl-82FujrFl2b2ftkvTv8Wgq5yKGQB7M",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 7,
    "prompt_tokens": 5,
    "total_tokens": 12
  },
  "warning": "This model version is deprecated. Migrate before January 4, 2024 to avoid disruption of service. Learn more https://platform.openai.com/docs/deprecations"
}


#### Prompt：解析返回结果，获取生成文本

```
下面是 openai completion api 的返回结果，将生成的内容存放在变量 text 中:

{
  "id": "cmpl-uqkvlQyYK7bGYrRHQ0eXlWi7",
  "object": "text_completion",
  "created": 1589478378,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\n\nThis is indeed a test",
      "index": 0,
      "logprobs": null,
      "finish_reason": "length"
    }
  ],
  "usage": {
    "prompt_tokens": 5,
    "completion_tokens": 7,
    "total_tokens": 12
  }
}
```

In [8]:
text = data['choices'][0]['text']
print(text)



This is indeed a test


### 生成中文文本

调整`prompt`, `max_tokens`, `temperatura` 等参数。

In [9]:
data = openai.Completion.create(
    model = "text-davinci-003",
    prompt = "使用中文，讲10个给程序员听得笑话",
    max_tokens = 1000,
    temperature = 0.5
)

text = data['choices'][0]['text']
print(text)



1. 两个程序员在聊天，第一个问：“你最近在学什么？”，第二个回答：“我在学习如何用一句话解释深度学习。”
2. 两个程序员在聊天，第一个问：“你最近在做什么？”，第二个回答：“我在把十个小时的工作压缩到一个小时。”
3. 一个程序员在和朋友聊天，朋友问：“你最近在做什么？”，程序员回答：“我在学习如何用更少的代码实现更多的功能。”
4. 一个程序员正在和朋友聊天，朋友问：“你最近在学什么？”，程序员回答：“我在学习如何用更少的睡眠实现更多的功能。”
5. 两个程序员在聊天，第一个问：“你最近在学什么？”，第二个回答：“我在学习如何用更少的代码实现更多的事情。”
6. 一个程序员正在和朋友聊天，朋友问：“你最近在做什么？”，程序员回答：“我在学习如何用更少的时间调试更多的程序。”
7. 两个程序员在聊天，第一个问：“你最近在做什么？”，第二个回答：“我在学习如何用更少的代码实现更多的事情，而且还要少花钱。”
8. 一个程序员正在和朋友聊天，朋友问：“你最近在做什么？”，程序员回答：“我在学习如何用更少的代码实现更多的功能，而且还要更快。”
9. 两个程


### 生成 Python 代码，并执行和验证

以面试中考察的典型的试题 `快速排序` 为例。

In [10]:
data = openai.Completion.create(
    model="text-davinci-003",
    prompt="生成可执行的快速排序 Python 代码",
    max_tokens=1000,
    temperature=0
)
text = data['choices'][0]['text']
print(text)



def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quick_sort(left) + middle + quick_sort(right)


#### Prompt：Jupyter Notebook 中执行生成的代码

Prompt：

```
我现在用 Completion API 生成了  Python 代码，并以字符串形式存放在 text 中，如下所示：

text = data['choices'][0]['text']
print(text)

def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quick_sort(left) + middle + quick_sort(right)

如何在 Jupyter notebook 中执行text中存放的这段代码
```

In [11]:
# `exec` 函数会执行传入的字符串作为 Python 代码。
# 在这个例子中，我们使用 `exec` 来定义了一个 `quick_sort` 函数，然后你就可以调用这个函数了。
# 请注意，`exec` 可以执行任何 Python 代码，因此在使用它的时候一定要小心，特别是当你执行的代码来自不可信的来源时。
exec(text)

print(quick_sort([12,3,6,8,10,1,2,1]))

[1, 1, 2, 3, 6, 8, 10, 12]


## Chat Completions API

使用 Chat Completions API 实现对话任务

聊天补全(Chat Completions API)以消息列表作为输入，并返回模型生成的消息作为输出。尽管聊天格式旨在使多轮对话变得简单，但它同样适用于没有任何对话的单轮任务。

主要请求参数说明（不包含 `function_call` 相关内容）：

- **`model` (string, Required)**

  ID of the model to use. See the [model endpoint compatibility](https://platform.openai.com/docs/models/model-endpoint-compatibility) table for details on which models work with the Chat API.

- **`messages` (array, Required)**
  
  A list of messages comprising the conversation so far. [Example Python code](https://cookbook.openai.com/examples/how_to_format_inputs_to_chatgpt_models).
    
    - **`role` (string, Required)**
      
      The role of the messages author. One of `system`, `user`, or `assistant`.

    - **`content` (string or null, Required)**
    
      The contents of the message. `content` is required for all messages, and may be null for assistant messages with function calls.
    
    - **`name` (string, Optional)**
    
      The name of the author of this message. May contain a-z, A-Z, 0-9, and underscores, with a maximum length of 64 characters.

- **`stream` (boolean or null, Optional, Defaults to false)**

  If set, partial message deltas will be sent, like in ChatGPT. Tokens will be sent as data-only [server-sent events](https://developer.mozilla.org/en-US/docs/Web/API/Server-sent_events/Using_server-sent_events#event_stream_format) as they become available, with the stream terminated by a `data: [DONE]` message. [Example Python code](https://cookbook.openai.com/examples/how_to_stream_completions).

- **`max_tokens` (integer or null, Optional, Defaults to inf)**

  The maximum number of [tokens](https://platform.openai.com/tokenizer) to generate in the chat completion.

  The total length of input tokens and generated tokens is limited by the model's context length. [Example Python code](https://cookbook.openai.com/examples/how_to_count_tokens_with_tiktoken) for counting tokens.

- **`temperature` (number, Optional, Defaults to 1)**

  What sampling temperature to use, between 0 and 2. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic.

  We generally recommend altering this or `top_p` but not both.

- **`top_p` (number or null, Optional, Defaults to 1)**

  An alternative to sampling with temperature, called nucleus sampling, where the model considers the results of the tokens with top_p probability mass. So 0.1 means only the tokens comprising the top 10% probability mass are considered.

  We generally recommend altering this or `temperature` but not both.

### 对话请求

In [12]:
messages=[
    {
        "role": "user", 
        "content": "Hello!"
    }
]

response = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = messages
)

print(response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Hi there! How can I assist you today?",
        "role": "assistant"
      }
    }
  ],
  "created": 1695548547,
  "id": "chatcmpl-82Fv9cbzdsqRCwf7setbIrAuFOibd",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 10,
    "prompt_tokens": 9,
    "total_tokens": 19
  }
}


#### Prompt：解析返回结果，获取生成文本，并维护聊天记录

```
我希望维护一个保存聊天记录的 messages 变量，如下所示：

messages=[
    {
        "role": "user", 
        "content": "Hello!"
    },
    {
        "content": "Hi! How can I assist you today?",
        "role": "assistant"
    },
]

下面是 openai chat completion api 的返回结果 ，将生成的 message 追加到 messages 结尾:

{
  choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Hi! How can I assist you today?",
        "role": "assistant"
      }
    }
  ],
  "created": 1690106747,
  "id": "chatcmpl-7fQGBBKSXTye4HsKcKMWKFVW63qTT",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 9,
    "prompt_tokens": 9,
    "total_tokens": 18
  }
}
```

In [13]:
new_message = response["choices"][0]["message"]
print(type(new_message))
print(new_message)

<class 'openai.openai_object.OpenAIObject'>
{
  "content": "Hi there! How can I assist you today?",
  "role": "assistant"
}


#### Prompt: OpenAIObject -> Dict

```
打印 messages 列表后发现数据类型不对，messages 输出如下：

print(messages)

[{'role': 'user', 'content': 'Hello!'}, <OpenAIObject at 0x7f27582c13f0> JSON: {
  "content": "Hello! How can I assist you today?",
  "role": "assistant"
}]

将OpenAIObject 转换为一个如下数据类型格式：

    {
        "role": "user", 
        "content": "Hello!"
    }
```

In [14]:
new_message_dict = json.loads(str(new_message))

# 将新的消息字典添加到 messages 列表
messages.append(new_message_dict)

pprint(messages)

[{'content': 'Hello!', 'role': 'user'},
 {'content': 'Hi there! How can I assist you today?', 'role': 'assistant'}]


### 新一轮对话

In [15]:
new_chat = {
    "role": "user",
    "content": "1.讲一个程序员才听得懂的冷笑话；2.今天是几号？3.明天星期几？"
}

messages.append(new_chat)
pprint(messages)

[{'content': 'Hello!', 'role': 'user'},
 {'content': 'Hi there! How can I assist you today?', 'role': 'assistant'},
 {'content': '1.讲一个程序员才听得懂的冷笑话；2.今天是几号？3.明天星期几？', 'role': 'user'}]


In [16]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages
)

new_message = response['choices'][0]['message']
# 打印 new_messages 
print(new_message)

{
  "content": "1. \u51b7\u7b11\u8bdd\uff1a\u6709\u4e24\u79cd\u4eba\uff0c\u4e00\u79cd\u662f\u61c2\u4e8c\u8fdb\u5236\u7684\uff0c\u53e6\u4e00\u79cd\u662f\u4e0d\u61c2\u7684\u3002\n\n2. \u4eca\u5929\u662f\u4ec0\u4e48\u65e5\u671f\uff1f\u5bf9\u4e0d\u8d77\uff0c\u6211\u65e0\u6cd5\u63d0\u4f9b\u5b9e\u65f6\u65e5\u671f\u3002\u60a8\u53ef\u4ee5\u67e5\u770b\u60a8\u7684\u8bbe\u5907\u4e0a\u7684\u65e5\u5386\u6216\u8005\u641c\u7d22\u5f15\u64ce\u6765\u83b7\u5f97\u5f53\u524d\u65e5\u671f\u3002\n\n3. \u660e\u5929\u661f\u671f\u51e0\uff1f\u540c\u6837\uff0c\u6211\u65e0\u6cd5\u63d0\u4f9b\u5b9e\u65f6\u65e5\u671f\u4fe1\u606f\u3002\u60a8\u53ef\u4ee5\u53c2\u8003\u65e5\u5386\u6216\u8005\u641c\u7d22\u5f15\u64ce\u6765\u83b7\u53d6\u660e\u5929\u7684\u661f\u671f\u51e0\u3002",
  "role": "assistant"
}


#### Prompt: 解析中文结果

```
新一轮对话的返回结果没有解析成中文，内容如下：

# 打印 new_message
print(new_message)

{
  "content": "1. \u51b7\u7b11\u8bdd\uff1a\u4e3a\u4ec0\u4e48\u7a0b\u5e8f\u5458\u559c\u6b22\u5728\u5ba4\u5916\u6563\u6b65\uff1f\u56e0\u4e3a\u201c\u5916\u9762\u201d\u662f\u4e00\u4e2a\u4e0d\u4f1a\u6709bug\u7684\u5730\u65b9\uff01\n\n2. \u4eca\u5929\u7684\u65e5\u671f\u662f\u6839\u636e\u60a8\u6240\u5728\u7684\u65f6\u533a\u800c\u5b9a\uff0c\u53ef\u4ee5\u901a\u8fc7\u67e5\u770b\u7535\u8111\u7684\u4efb\u52a1\u680f\u6216\u624b\u673a\u7684\u65e5\u671f\u65f6\u95f4\u8bbe\u7f6e\u6765\u786e\u8ba4\u3002\n\n3. \u660e\u5929\u7684\u661f\u671f\u51e0\u4e5f\u662f\u6839\u636e\u60a8\u6240\u5728\u7684\u65f6\u533a\u800c\u5b9a\uff0c\u60a8\u53ef\u4ee5\u4f7f\u7528\u65e5\u5386\u5e94\u7528\u6216\u901a\u8fc7\u641c\u7d22\u5f15\u64ce\u8f93\u5165\"\u660e\u5929\u662f\u661f\u671f\u51e0\"\u6765\u83b7\u5f97\u51c6\u786e\u7684\u7b54\u6848\u3002",
  "role": "assistant"
}
```

In [17]:
# GPT-4: 从你提供的信息中看，这段文本是经过Unicode编码的中文。
# 你可以使用Python的print()函数将其打印出来，以获取原始的中文字符串：
print(new_message.content)

1. 冷笑话：有两种人，一种是懂二进制的，另一种是不懂的。

2. 今天是什么日期？对不起，我无法提供实时日期。您可以查看您的设备上的日历或者搜索引擎来获得当前日期。

3. 明天星期几？同样，我无法提供实时日期信息。您可以参考日历或者搜索引擎来获取明天的星期几。


### System 对象

提供预设的系统级别的 prompt，有助于更准确的信息生成。

In [18]:
# 构造聊天记录
messages = [
    {"role": "system", "content": "你是一个乐于助人的体育界专家。"},
    {"role": "user", "content": "2008年奥运会是在哪里举行的？"},
]

data = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = messages
)

message = data["choices"][0]["message"]["content"]
messages.append({"role": "assistant", "content": message})
print(message)

2008年奥运会在中国的北京市举行。


#### 新一轮对话(有上下文)

In [20]:
messages.append({"role": "user", "content": "1.金牌最多的是哪个国家？2.奖牌最多的是哪个国家？"})

data = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = messages
)

message = data["choices"][0]["message"]["content"]
messages.append({"role": "assistant", "content": message})
print(message)

1. 2008年奥运会金牌最多的国家是中国，共获得51枚金牌。
2. 2008年奥运会奖牌最多的国家是美国，共获得110枚奖牌，其中包括36枚金牌、38枚银牌和36枚铜牌。


#### 全新对话(无上下文)

In [21]:
data = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{'role': 'user', 'content': '1.金牌最多的是哪个国家？2.奖牌最多的是哪个国家？'}]
)

message = data["choices"][0]["message"]["content"]
print(message)

1.金牌最多的国家是美国。
2.奖牌最多的国家是美国。
